In [ ]:
import os
CFG = {
    'model_arch': 'tf_efficientnet_b4_ns',
    'img_size': 512,
    'num_classes': 5,
    'tta': 5,
    'savecsv': "tmpfilefold/pseudo_submission.csv",
    'srcImgPath':'../input/cassava-leaf-disease-classification/test_images',
    'weights': [1],
    'models':['EB4.pth.tar'],
    'baseModelPath':'../input/pseudotraining',
    'base_save_path':'./tmpfilefold/cassava-pseudo',
    'saveScores':0.4,
    
}

In [ ]:
if os.path.exists('./tmpfilefold'):
    !rm -r ./tmpfilefold
if not os.path.exists('./tmpfilefold'):
    os.mkdir('./tmpfilefold')
if not os.path.exists(CFG['base_save_path']):
    os.mkdir(CFG['base_save_path'])

In [ ]:
from tqdm import tqdm
import numpy as np
def inference_one_epoch(model, data_loader):
    model.eval()
    image_preds_all = []
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for i,(img,imgpath) in pbar:
        img = img.cuda()
        with torch.no_grad():
            image_preds = model(img)
            image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [ ]:
import sys
import cv2
import argparse
import pandas as pd
import os
from torch.utils.data import Dataset,DataLoader
import albumentations as A
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from albumentations.pytorch.transforms import ToTensor
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)
from albumentations.pytorch import ToTensorV2
import torch
def test_transform(width = 512,height = 512):
    return A.Compose([A.CLAHE(),
                       A.Resize(height = height,width = width,p = 1.0),
                       ToTensor(sigmoid=False,normalize={'mean':[0.485, 0.456, 0.406],'std':[0.229, 0.224, 0.225]})]
                       )

def test_transform_v2(width = 512,height = 512):
    return Compose([
#             CenterCrop(width, height, p=1.),
            Resize(width, height,p=1.0),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

def test_transform_v3():
    return Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

def test_transform_v4():
    return Compose([
            Resize(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)


class LoadImagesAndLabels(Dataset):  # for training/testing
    def __init__(self,baseImgPath = '', trans = None):
        self.image_ids = []
        self.baseImgPath = baseImgPath
        for imgname in os.listdir(self.baseImgPath):
            self.image_ids.append(os.path.join(self.baseImgPath,imgname))
        self.trans = trans

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, index):
        imgpath = self.image_ids[index]
        img = cv2.imread(imgpath)
        assert img is not None,imgpath
        img = img[:, :, ::-1]
        img = self.trans(image = img)['image']
        imgname = imgpath[imgpath.rfind('/')+1:]
        return img,imgname

In [ ]:
import sys
sys.path.insert(0,'../input/pseudotraining/pytorch-image-models')
from timm import create_model
import torch
from itertools import compress

def makePseudolabel():
    args = CFG
    test = pd.DataFrame()
    model = create_model(model_name = args['model_arch'],num_classes=args['num_classes'])
    trans = test_transform_v4()
    val_dataset = LoadImagesAndLabels(args['srcImgPath'],trans)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=60, shuffle=False,num_workers=12, pin_memory=True)
    tst_preds = []
    with torch.no_grad():
        for i,model_name in enumerate(CFG['models']):
            model_path = os.path.join(args['baseModelPath'],model_name)
            print('model_path: ',model_path)
            state_dict = torch.load(model_path,map_location='cpu')
            model.load_state_dict(state_dict["state_dict"], strict=True)
            model.cuda()
            for _ in range(CFG['tta']):
                tst_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(model, val_loader)]
    tst_preds = np.sum(tst_preds, axis=0)
    tst_preds_max = np.max(tst_preds,axis = -1)
    index = tst_preds_max > args['saveScores']
    test['image_id'] = list(compress(list(os.listdir(args['srcImgPath'])), list(index)))
    test['label'] = np.argmax(tst_preds, axis=1)[index]
    test.head()
    test.to_csv(args['savecsv'], index=False)
    print('makePseudolabel Done!')

In [ ]:
def coptfile():
    import csv
    import os
    import shutil
    args = CFG
    test = pd.DataFrame()
    csv_reader = csv.reader(open(args['savecsv']))
    label_dict = {'0':'CBB', '1':'CBSD', '2':"CGM", '3':"CMD",'4':"Healthy"}
    count = {'CBB':0,'CBSD':0,"CGM":0,"CMD":0,"Healthy":0}
    for line in csv_reader:
        if line[0].find('.jpg') == -1:
            continue
        img_name = line[0]
        label = line[1]
        img_short_dir = label_dict[label]
        dst_path = os.path.join(args['base_save_path'],img_name) 
        src_path = os.path.join(args['srcImgPath'],img_name)
        count[img_short_dir] += 1
        shutil.copyfile(src_path,dst_path)
    print('coptfile Done!')

In [ ]:
import csv
import cv2
import sys
import numpy as np
import torch
sys.path.insert(0,'../input/pseudotraining')
args = CFG
num_imgs = len(list(os.listdir(args['srcImgPath'])))
if num_imgs >= 10:
    makePseudolabel()
    coptfile()
    num_saveImgs = len(list(os.listdir(args['base_save_path'])))
    if num_saveImgs >= 10:
        csv.reader(open('./tmpfilefold/pseudo_submission.csv'))
        torch.cuda.empty_cache()
        !python ../input/pseudotraining/pytorch-image-models/train_pseudo.py --data './tmpfilefold/cassava-pseudo' --cvsPath './tmpfilefold/pseudo_submission.csv' --initial-checkpoint '../input/pseudotraining/EB4.pth.tar' --output 'tmpfilefold' --batch-size 10
else:
    print('NO PSEUDO!')

In [ ]:
torch.cuda.empty_cache()
CFG = {
    'model_arch': 'tf_efficientnet_b4_ns',
    'img_size': 512,
    'num_classes': 5,
    'tta': 5,
    'srcImgPath':'../input/cassava-leaf-disease-classification/test_images',
    'weights': [1],
    'pseudo_model_path': './tmpfilefold',
    'base_model_path': './tmpfilefold/cassava-pseudo'
}
if os.path.exists(os.path.join(CFG['pseudo_model_path'],'model_best.pth.tar')):
    CFG['models'] = ['model_best.pth.tar','EB4.pth.tar']
    CFG['base_model_path'] = CFG['pseudo_model_path']
    CFG['weights'] = [1,1] 
else:
    CFG['models'] = ['EB4.pth.tar']

print(CFG['models'])

In [ ]:
from timm import create_model
import torch

args = CFG
test = pd.DataFrame()
test['image_id'] = list(os.listdir(args['srcImgPath']))
model = create_model(model_name = args['model_arch'],num_classes=args['num_classes'])
trans = test_transform_v4()
val_dataset = LoadImagesAndLabels(args['srcImgPath'],trans)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=60, shuffle=False,num_workers=16, pin_memory=True)
tst_preds = []
with torch.no_grad():
    for i,model_name in enumerate(CFG['models']):
        if model_name.find('model_best') != -1:
            model_path = os.path.join(CFG['base_model_path'],model_name)
        else:
            model_path = os.path.join('../input/pseudotraining',model_name)
        print('model_path: ',model_path)
        state_dict = torch.load(model_path,map_location='cpu')
        model.load_state_dict(state_dict["state_dict"], strict=True)
        model.cuda()
        for _ in range(CFG['tta']):
            tst_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(model, val_loader)]
tst_preds = np.sum(tst_preds, axis=0)
test['label'] = np.argmax(tst_preds, axis=1)
test.head()
test.to_csv('submission.csv', index=False)
print('Done!')
if os.path.exists('./tmpfilefold'):
    !rm -r ./tmpfilefold